## Local RAG

This is the local RAG system that Marco showcased during the last session of Meeting&nbsp;2. It requires you to install [Ollama](https://www.ollama.com/) and download the `nomic-text-embed` (for querying) and `llama-3` (for generating queries and answering questions) models.

In [1]:
!pip install --q unstructured langchain
!pip install --q "unstructured[all-docs]"

In [2]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [3]:
local_path = "some.pdf"    # insert your PDF filename here

In [4]:
loader = UnstructuredPDFLoader(file_path=local_path)
data = loader.load()

In [5]:
data[0].page_content

'001\n\n002\n\n003\n\n004\n\n005\n\n006\n\n007\n\n008\n\n009\n\n010\n\n011\n\n012\n\n013\n\n014\n\n015\n\n016\n\n017\n\n018\n\n019\n\n020\n\n021\n\n022\n\n023\n\n024\n\n025\n\n026\n\n027\n\n028\n\n029\n\n030\n\n031\n\n032\n\n033\n\n034\n\n035\n\n036\n\n037\n\n038\n\n039\n\n040\n\nPost-Project Paper, TDDD09\n\nJoline Hellström\n\n1 Describe\n\nIn our project, we explored a method called back- translation and investigated its impact on machine translation. Moreover, we examined if the effi- ciency of back-translation depends on how closely related the source and target language are. To in- vestigate this, we trained models to translate from Swedish to the respective target languages Nor- wegian, Finnish, and Northern Sámi (also known as Davvisámegiella). Meanwhile Norwegian is closely related to Swedish, neither Finnish nor Northern Sámi comes from the same family of languages as Swedish.\n\nmodel that translates from Swedish to the target lan- guage and a reverse model that translates f

In [6]:
!pip install --q chromadb
!pip install --q langchain-text-splitters

In [7]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [26]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [27]:
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model='nomic-embed-text', show_progress=True),
    collection_name='local-rag',
)

OllamaEmbeddings: 100%|███████████████████████████| 4/4 [00:01<00:00,  2.73it/s]


In [28]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core. runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [29]:
local_model = 'llama3'
llm = ChatOllama(model=local_model)

In [30]:
# Based on the prompt format for Llama 3; see their documentation for details

def make_prompt(message):
    return f'<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{message}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'

In [31]:
QUERY_PROMPT = PromptTemplate(
    input_variables = ['question'],
    template = make_prompt('You are an AI language model assistant. Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search. Provide these alternative questions separated by newlines.\n\nOriginal question: {question}'),
)

In [32]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm,
    prompt=QUERY_PROMPT,
)

In [33]:
template = make_prompt("Answer the question based ONLY on the following context:\n\n{context}\n\nQuestion: {question}")

prompt = ChatPromptTemplate.from_template(template)

In [34]:
chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [35]:
chain.invoke('What is this document about?')

OllamaEmbeddings: 100%|███████████████████████████| 1/1 [00:00<00:00, 62.01it/s]


'Based on the provided context, this document appears to be a thesis or research paper in the field of Natural Language Processing (NLP). Specifically, it discusses the application of machine translation models, particularly transformer models, to translate text from Swedish to Northern Sámi. The document also touches on topics such as preprocessing data, attention mechanisms in transformers, and the challenges of working with low-resource language settings.'

In [21]:
chain.invoke('What languages are mentioned in the paper?')

OllamaEmbeddings: 100%|███████████████████████████| 1/1 [00:00<00:00, 46.71it/s]


'According to the context, the following languages are mentioned:\n\n1. Swedish\n2. Northern Sámi\n3. Norwegian\n4. Finnish'

In [22]:
chain.invoke('What is the BLEU score of the final Swedish-Sámi model?')

OllamaEmbeddings: 100%|███████████████████████████| 1/1 [00:00<00:00, 51.44it/s]


"According to the text, the results from the evaluation for each language pair's baseline and final model are presented in Table 1. For the Swedish-Sámi model, the BLEU score of the final model is:\n\n24.35"

In [25]:
chain.invoke('List the main learnings articulated by the author.')

OllamaEmbeddings: 100%|███████████████████████████| 1/1 [00:00<00:00, 40.87it/s]


"Based on the provided context, the main learnings articulated by the author are:\n\n1. The importance of quality in machine translation, as highlighted by the BLEU scores and the difficulty in evaluating the performance of the model.\n2. The value of preprocessing data to prepare it for training, including techniques such as removing duplicate sentences and using byte-pair-encoding and stemming.\n3. The transformer architecture's ability to utilize attention mechanisms to allow models to consider how words relate to each other in a sentence during translation.\n4. The complexity and resource requirements of the transformer model, including its large number of parameters and need for long training times.\n\nThese learnings were gained through hands-on experience with the openNMT framework and by reading papers on machine translation, such as Stenlund et al. (2023) and Vaswani et al. (2023)."